# 04 model experiments - TODO

## Purpose
To be filled during project development

## Steps
1. 
2. 
3. 

In [1]:
# CELL 1: All imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import json

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print("🧠 SIMPLE LSTM Model for EUR/USD Prediction")
print("="*50)

🧠 SIMPLE LSTM Model for EUR/USD Prediction


In [2]:
# CELL 1: Imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import warnings
warnings.filterwarnings('ignore')


import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
print("✅ TensorFlow/Keras imported successfully")

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Path setup
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

DATA_PROCESSED = os.path.join(project_root, 'data', 'processed')
FIGURES = os.path.join(project_root, 'figures')
MODELS = os.path.join(project_root, 'models')

print("🧠 SIMPLE LSTM Model for EUR/USD Prediction")
print("="*50)


✅ TensorFlow/Keras imported successfully
🧠 SIMPLE LSTM Model for EUR/USD Prediction


In [3]:

# CELL 2: Load prepared data
print("\n📂 Loading prepared sequences...")

# Check if files exist
required_files = [
    os.path.join(DATA_PROCESSED, 'X_train.npy'),
    os.path.join(DATA_PROCESSED, 'y_train.npy'),
    os.path.join(DATA_PROCESSED, 'X_test.npy'),
    os.path.join(DATA_PROCESSED, 'y_test.npy'),
    os.path.join(MODELS, 'lstm_scaler.joblib')
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print(f"❌ Missing files: {missing_files}")
    print("Please run notebook 3 first!")
    raise FileNotFoundError("Required files missing")

# Load the data
X_train = np.load(os.path.join(DATA_PROCESSED, 'X_train.npy'))
y_train = np.load(os.path.join(DATA_PROCESSED, 'y_train.npy'))
X_test = np.load(os.path.join(DATA_PROCESSED, 'X_test.npy'))
y_test = np.load(os.path.join(DATA_PROCESSED, 'y_test.npy'))
scaler = joblib.load(os.path.join(MODELS, 'lstm_scaler.joblib'))

print(f"✅ Data loaded:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  X_test:  {X_test.shape}")
print(f"  y_test:  {y_test.shape}")



📂 Loading prepared sequences...
✅ Data loaded:
  X_train: (445, 100, 1)
  y_train: (445,)
  X_test:  (134, 100, 1)
  y_test:  (134,)


In [4]:

# CELL 3: Build SIMPLER LSTM model (if Linear Regression beats complex LSTM)
print("\n🏗️ Building SIMPLER LSTM model...")
print("(Since complex LSTM performed worse than Linear Regression)")


model = Sequential([
    # First LSTM layer
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    
    # Second LSTM layer
    LSTM(units=50, return_sequences=False),  # Don't return sequences
    Dropout(0.2),
    
    # Dense layers
    Dense(units=25, activation='relu'),
    Dropout(0.2),
    
    # Output layer
    Dense(units=1)
])

# Compile with different optimizer if needed
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

print("✅ Simple LSTM model built")
print("\nModel Summary:")
model.summary()



🏗️ Building SIMPLER LSTM model...
(Since complex LSTM performed worse than Linear Regression)
✅ Simple LSTM model built

Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 50)        │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 25)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,901 (124.61 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# CELL 4: Train the model
print("\n🚀 Training LSTM model...")

# # Simple callbacks
# callbacks = [
#     EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
#     ModelCheckpoint(
#         filepath=os.path.join(MODELS, 'lstm_simple_best.h5'),
#         monitor='val_loss',
#         save_best_only=True,
#         verbose=0
#     )
# ]

# Train with fewer epochs since we have EarlyStopping
history = model.fit(
    X_train, y_train,
    epochs=30,  # Reduced from 50
    batch_size=32,
    validation_split=0.2,
    verbose=1,
    # callbacks=callbacks
)


print("✅ Training completed!")



🚀 Training LSTM model...
Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.1360 - mae: 0.3099 - val_loss: 0.0232 - val_mae: 0.1329
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0405 - mae: 0.1592 - val_loss: 0.0181 - val_mae: 0.1150
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0276 - mae: 0.1335 - val_loss: 0.0177 - val_mae: 0.1113
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0249 - mae: 0.1244 - val_loss: 0.0188 - val_mae: 0.1180
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0248 - mae: 0.1266 - val_loss: 0.0173 - val_mae: 0.1104
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0212 - mae: 0.1164 - val_loss: 0.0186 - val_mae: 0.1174
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0211 - mae: 0.1161 - val_loss: 0.0186 - val_mae: 0.1175
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0224 - mae: 0.1204 - val_loss: 0.0153 - val_mae: 0.1020
Epoch 9/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
# CELL 6: Make predictions
print("\n🔮 Making predictions on test set...")

y_pred_scaled = model.predict(X_test)

# Inverse transform to get actual prices
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_original = scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))

print(f"Made {len(y_pred_original)} predictions")



🔮 Making predictions on test set...
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Made 134 predictions


In [8]:
# CELL 7: Calculate metrics
print("\n📊 Evaluating model performance...")

mae = mean_absolute_error(y_test_original, y_pred_original)
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
r2 = r2_score(y_test_original, y_pred_original)

print("LSTM Model Performance:")
print(f"  MAE:  ${mae:.4f}")
print(f"  RMSE: ${rmse:.4f}")
print(f"  R²:   {r2:.4f}")



📊 Evaluating model performance...
LSTM Model Performance:
  MAE:  $0.0189
  RMSE: $0.0199
  R²:   -3.4407


In [9]:
# CELL 8: Save the model (THIS IS THE IMPORTANT PART!)
print("\n💾 Saving model and results...")

# 1. Save the trained model
model_path = os.path.join(MODELS, 'lstm_trained_model.keras')
model.save(model_path)
print(f"✅ Model saved to: {model_path}")

# 2. Save predictions
predictions_df = pd.DataFrame({
    'actual': y_test_original.flatten(),
    'predicted': y_pred_original.flatten(),
    'error': y_test_original.flatten() - y_pred_original.flatten()
})

predictions_path = os.path.join(DATA_PROCESSED, 'lstm_predictions.csv')
predictions_df.to_csv(predictions_path, index=False)
print(f"✅ Predictions saved to: {predictions_path}")

# 3. Save metrics
metrics = {
    'mae': float(mae),
    'rmse': float(rmse),
    'r2': float(r2),
    'training_samples': len(X_train),
    'testing_samples': len(X_test),
    'model_architecture': 'LSTM-50-50-Dense25',
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

metrics_path = os.path.join(MODELS, 'lstm_metrics.json')
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"✅ Metrics saved to: {metrics_path}")


💾 Saving model and results...
✅ Model saved to: /Users/guillermo/Documents/Ironhack/M8_Capstone/2-Capstone-2_MLOps/1-Development/eurusd-capstone/models/lstm_trained_model.keras
✅ Predictions saved to: /Users/guillermo/Documents/Ironhack/M8_Capstone/2-Capstone-2_MLOps/1-Development/eurusd-capstone/data/processed/lstm_predictions.csv
✅ Metrics saved to: /Users/guillermo/Documents/Ironhack/M8_Capstone/2-Capstone-2_MLOps/1-Development/eurusd-capstone/models/lstm_metrics.json


In [10]:
# CELL 9: Make a future prediction
print("\n🔮 Making next day prediction...")

# Load the full dataset to get recent prices
train_data = pd.read_csv(os.path.join(DATA_PROCESSED, 'lstm_simple_train_data.csv'), 
                        index_col=0, parse_dates=True)
test_data = pd.read_csv(os.path.join(DATA_PROCESSED, 'lstm_simple_test_data.csv'),
                       index_col=0, parse_dates=True)

all_data = pd.concat([train_data, test_data])

# Get last 100 days (sequence length from notebook 3)
recent_prices = all_data['close'].values[-100:]

# Scale using our scaler
recent_scaled = scaler.transform(recent_prices.reshape(-1, 1))

# Reshape for LSTM: (1 sample, 100 timesteps, 1 feature)
last_sequence = recent_scaled.reshape(1, 100, 1)

# Predict
next_day_scaled = model.predict(last_sequence)
next_day_price = scaler.inverse_transform(next_day_scaled)[0][0]

current_price = recent_prices[-1]
predicted_change = ((next_day_price - current_price) / current_price) * 100

print(f"Current EUR/USD price: ${current_price:.4f}")
print(f"Predicted next day: ${next_day_price:.4f}")
print(f"Predicted change: {predicted_change:+.2f}%")

# Save the future prediction
future_pred = {
    'current_price': float(current_price),
    'predicted_next_day': float(next_day_price),
    'predicted_change_pct': float(predicted_change),
    'prediction_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

future_path = os.path.join(MODELS, 'lstm_future_prediction.json')
with open(future_path, 'w') as f:
    json.dump(future_pred, f, indent=2)
print(f"✅ Future prediction saved to: {future_path}")

# CELL 10: Summary
print("\n" + "=" * 60)
print("✅ LSTM MODEL TRAINING COMPLETE")
print("=" * 60)

print(f"\n📊 MODEL PERFORMANCE:")
print(f"  • MAE: ${mae:.4f}")
print(f"  • R²:  {r2:.4f}")

print(f"\n💾 FILES SAVED:")
print(f"  • Model: {model_path}")
print(f"  • Predictions: {predictions_path}")
print(f"  • Metrics: {metrics_path}")
print(f"  • Future prediction: {future_path}")

print(f"\n🔮 NEXT DAY PREDICTION:")
print(f"  • Current: ${current_price:.4f}")
print(f"  • Predicted: ${next_day_price:.4f}")
print(f"  • Change: {predicted_change:+.2f}%")

print(f"\n🚀 NEXT STEP:")
print("  Open notebook 5 to compare LSTM with traditional ML models")

# Verify the model file was created
if os.path.exists(model_path):
    file_size = os.path.getsize(model_path) / 1024  # Convert to KB
    print(f"\n✅ Verification: Model file exists ({file_size:.1f} KB)")
else:
    print(f"\n❌ ERROR: Model file was not created at {model_path}")


🔮 Making next day prediction...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Current EUR/USD price: $1.1641
Predicted next day: $1.1459
Predicted change: -1.56%
✅ Future prediction saved to: /Users/guillermo/Documents/Ironhack/M8_Capstone/2-Capstone-2_MLOps/1-Development/eurusd-capstone/models/lstm_future_prediction.json

✅ LSTM MODEL TRAINING COMPLETE

📊 MODEL PERFORMANCE:
  • MAE: $0.0189
  • R²:  -3.4407

💾 FILES SAVED:
  • Model: /Users/guillermo/Documents/Ironhack/M8_Capstone/2-Capstone-2_MLOps/1-Development/eurusd-capstone/models/lstm_trained_model.keras
  • Predictions: /Users/guillermo/Documents/Ironhack/M8_Capstone/2-Capstone-2_MLOps/1-Development/eurusd-capstone/data/processed/lstm_predictions.csv
  • Metrics: /Users/guillermo/Documents/Ironhack/M8_Capstone/2-Capstone-2_MLOps/1-Development/eurusd-capstone/models/lstm_metrics.json
  • Future prediction: /Users/guillermo/Documents/Ironhack/M8_Capstone/2-Capstone-2_MLOps/1-Development/eurusd-capstone/models/lstm_future_prediction.json